### Advanced in silico drug design workshop. Olomouc, 21-25 January, 2019.
### Deep Leaning Tutorial: Multi-Layer Perceptron with Keras

Dr Thomas Evangelidis

Research Scientist

IOCB - Institute of Organic Chemistry and Biochemistry of the Czech Academy of Sciences
Prague, Czech Republic

  & 
CEITEC - Central European Institute of Technology
Brno, Czech Republic 

email: tevang3@gmail.com

website: https://sites.google.com/site/thomasevangelidishomepage/


##Objectives:

In this tutorial you will learn how to construct a simple Multi-Layer Perceptron model with Keras. Specifically you will learn to:
* Create and add layers including weight initialization and activation.
* Compile models including optimization method, loss function and metrics.
* Fit models include epochs and batch size.
* Model predictions.
* Summarize the model.
* Train an initial model on large relevant data and transfer the hidden layers of that model to a new one, which will be training with fewer focused data (Transfer LEarning).

In [104]:
from keras.wrappers.scikit_learn import KerasRegressor, KerasClassifier
from keras.callbacks import EarlyStopping
from rdkit.Chem import AllChem, SDMolSupplier
from rdkit import DataStructs
import numpy as np
from keras.models import Sequential, Input, Model
from keras.layers import Dense
from scipy.stats import kendalltau, pearsonr
from sklearn.metrics import mean_squared_error, roc_auc_score
from sklearn.model_selection import cross_validate, KFold

#### Reading molecules and activity from SDF

In [150]:
fname = "data/cdk2.sdf"

mols = []
y = []
for mol in SDMolSupplier(fname):
    if mol is not None:
        mols.append(mol)
        y.append(float(mol.GetProp("pIC50")))

#### Calculate descriptors (fingerprints) and convert them into numpy array

In [151]:
# generate binary Morgan fingerprint with radius 2
fp = [AllChem.GetMorganFingerprintAsBitVect(m, 2) for m in mols]

In [152]:
def rdkit_numpy_convert(fp):
    output = []
    for f in fp:
        arr = np.zeros((1,))
        DataStructs.ConvertToNumpyArray(f, arr)
        output.append(arr)
    return np.asarray(output)

In [153]:
x = rdkit_numpy_convert(fp)

In [154]:
# fix random seed for reproducibility
seed = 2019
np.random.seed(seed)
mol_num, feat_num = x.shape
print("# molecules for training = %i, # of features = %i\n" % (mol_num, feat_num))

#### Define a function to create a simple feed forward network with one fully connected hidden layer or 300 neurons. The network uses the rectifier activation function for the hidden layer. No activation function is used for the output layer because it is a regression problem and we are interested in predicting numerical values directly without transform. The ADAM algorithm is employed to optimize the loss function.

In [155]:
# create model
def MLP1(feat_num, loss):
    net = Sequential()
    net.add(Dense(300, input_dim=feat_num, kernel_initializer='normal', activation='relu'))
    net.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    net.compile(loss=loss, optimizer='adam')
    return net

#### Print summary of layers and trainable parameters.

In [112]:
MLP1(feat_num, 'mean_squared_error').summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_208 (Dense)            (None, 300)               614700    
_________________________________________________________________
dense_209 (Dense)            (None, 1)                 301       
Total params: 615,001
Trainable params: 615,001
Non-trainable params: 0
_________________________________________________________________


#### Evaluate model with Keras wrapper of scikit-learn (faster and easier), using Mean Squared Error as the loss function, 300 training epochs and batch size 1/8 of the training set.

In [156]:
estimator = KerasRegressor(build_fn=MLP1, 
                           feat_num=feat_num, 
                           loss='mean_squared_error', 
                           epochs=300, 
                           batch_size=int(x.shape[0]/8), 
                           verbose=0)

#### Define our own evaluation metrics for the model: 1) Kendall's tau (ranking correlation), 2) Pearson's R (correlation), 3) Mean Squared Error. The evaluation will be done with 5-fold cross-validation.

In [157]:
def kendalls_tau(estimator, X, y):
    preds = estimator.predict(X)
    t = kendalltau(preds, y)[0]
    return t

def pearsons_r(estimator, X, y):
    preds = estimator.predict(X)
    r = pearsonr(preds, y)[0]
    return r

def MSE(estimator, X, y):
    preds = estimator.predict(X)
    mse = mean_squared_error(preds, y)
    return mse
    
    
scoring = {'tau': kendalls_tau, 'R':pearsons_r, 'MSE':MSE}
kfold = KFold(n_splits=5, random_state=seed)
scores = cross_validate(estimator, x, y, scoring=scoring, cv=kfold, return_train_score=False)
print(scores)
print("\nResults: average tau=%f+-%f, average R=%f+-%f, average MSE=%f+-%f\n" % 
      (scores['test_tau'].mean(), scores['test_tau'].std(),
       scores['test_R'].mean(), scores['test_R'].std(),
       scores['test_MSE'].mean(), scores['test_MSE'].std()))

{'score_time': array([0.72287917, 0.73120093, 0.73302698, 0.74567223, 0.74707484]), 'test_MSE': array([1.23098004, 1.85036503, 1.34946516, 0.74074917, 1.04380864]), 'fit_time': array([17.85927796, 18.00152087, 17.98357105, 17.95444679, 17.97064114]), 'test_R': array([0.61573605, 0.48036024, 0.71477564, 0.84383886, 0.83615917]), 'test_tau': array([0.40837709, 0.29654438, 0.37620499, 0.60010937, 0.5901813 ])}

Results: average tau=0.454283+-0.120680, average R=0.698174+-0.137675, average MSE=1.243074+-0.366689



#### Running this code gives us an estimate of the model’s performance on the problem for unseen data. The result reports the average and standard deviation of each metric across all 5 folds of the cross validation evaluation.

#### Now let's try Absolute Mean Error as a loss function.

In [115]:
estimator = KerasRegressor(build_fn=MLP1, 
                           feat_num=feat_num, 
                           loss='mean_absolute_error', 
                           epochs=300, 
                           batch_size=int(x.shape[0]/8),
                           verbose=0)

In [116]:
scores = cross_validate(estimator, x, y, scoring=scoring, cv=kfold, return_train_score=False)
print (scores)
print("\nResults: average tau=%f+-%f, average R=%f+-%f, average MSE=%f+-%f\n" % 
      (scores['test_tau'].mean(), scores['test_tau'].std(),
       scores['test_R'].mean(), scores['test_R'].std(),
       scores['test_MSE'].mean(), scores['test_MSE'].std()))

{'score_time': array([0.56212401, 0.57379794, 0.57216287, 0.58115292, 0.57715392]), 'test_MSE': array([1.41889084, 1.6827523 , 2.1521906 , 0.87549575, 0.86927765]), 'fit_time': array([17.00753903, 17.0233171 , 17.04589701, 17.09628987, 16.98412704]), 'test_R': array([0.60938719, 0.53293132, 0.62721648, 0.82024583, 0.86493694]), 'test_tau': array([0.40209436, 0.33190468, 0.33700571, 0.60975829, 0.63473341])}

Results: average tau=0.463099+-0.132514, average R=0.690944+-0.128587, average MSE=1.399721+-0.490483



#### We see a subtle performance increase.

#### Now let's add an extra hidden layer to the network with 200 neurons, to see if the performance improves further.

In [117]:
# create model
def MLP2(feat_num, loss):
    net = Sequential()
    net.add(Dense(300, input_dim=feat_num, kernel_initializer='normal', activation='relu'))
    net.add(Dense(200, input_dim=feat_num, kernel_initializer='normal', activation='relu'))
    net.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    net.compile(loss=loss, optimizer='adam')
    return net

#### Print summary of layers and trainable parameters.

In [118]:
MLP2(feat_num, 'mean_absolute_error').summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_230 (Dense)            (None, 300)               614700    
_________________________________________________________________
dense_231 (Dense)            (None, 200)               60200     
_________________________________________________________________
dense_232 (Dense)            (None, 1)                 201       
Total params: 675,101
Trainable params: 675,101
Non-trainable params: 0
_________________________________________________________________


#### We increase the training epochs to 500 because the addition of an extra layer increased the trainable variables.

In [119]:
estimator = KerasRegressor(build_fn=MLP2, 
                           feat_num=feat_num, 
                           loss='mean_absolute_error', 
                           epochs=500,
                           batch_size=int(x.shape[0]/8), 
                           verbose=0)

In [120]:
scores = cross_validate(estimator, x, y, scoring=scoring, cv=kfold, return_train_score=False)
print scores
print("\nResults: average tau=%f+-%f, average R=%f+-%f, average MSE=%f+-%f\n" % 
      (scores['test_tau'].mean(), scores['test_tau'].std(),
       scores['test_R'].mean(), scores['test_R'].std(),
       scores['test_MSE'].mean(), scores['test_MSE'].std()))

{'score_time': array([0.59869599, 0.591995  , 0.60312104, 0.60489297, 0.61515594]), 'test_MSE': array([1.27065902, 1.69945694, 2.60388686, 0.85742932, 0.85854965]), 'fit_time': array([29.43800306, 29.42567801, 29.50881195, 29.47552991, 29.63292408]), 'test_R': array([0.61633608, 0.55587068, 0.51790747, 0.82425183, 0.86172713]), 'test_tau': array([0.40575929, 0.36244313, 0.25657722, 0.59957331, 0.62077733])}

Results: average tau=0.449026+-0.140405, average R=0.675219+-0.141035, average MSE=1.457996+-0.652147



#### We don't see statistically significant improvement because our training set is small (436 molecules).

# Transfer Learning

#### We will use all binding assays for CDK2 from CHEMBL (1519 molecules) to train a network and then we will transfer its hidden layer to a new network which we shall train with the smaller training set that we have been using so far.

#### The following block of code is just for data preparation,  you can go hrough it very fast.

In [121]:
# LOAD TRAINING DATA FOR TRANSFER LEARNING
fname = "data/cdk2_large.sdf"

mols = []
molnames = []
for mol in SDMolSupplier(fname):
    if mol is not None:
        molname = mol.GetProp("_Name")
        if not molname in molnames:
            molnames.append(molname)
            mols.append(mol)

In [122]:
molname2pK_dict = {}
with open("data/cdk2_pK.dat", 'r') as f:
    for line in f:
        molname, pK = line.split()
        if not molname in molname2pK_dict.keys():
            molname2pK_dict[molname] = float(pK)

In [123]:
molnames1 = set(molnames)
molnames2 = set(molname2pK_dict.keys())
common_molnames = molnames1.intersection(molnames2)
y_transf = [molname2pK_dict[molname] for molname in molnames if molname in common_molnames]

#### Generate binary Morgan fingerprint with radius 2 as feature vectors for training.

In [124]:
fp = [AllChem.GetMorganFingerprintAsBitVect(m, 2) for m in mols if m.GetProp("_Name") in common_molnames]
x_transf = rdkit_numpy_convert(fp)

In [125]:
mol_num, feat_num = x_transf.shape
print("# molecules for transfer training = %i, # of features = %i\n" % (mol_num, feat_num))

# molecules for transfer training = 1519, # of features = 2048



#### We train a network with one fully connected hidden layer of 300 neurons, like in our first example.

In [126]:
net = MLP1(feat_num=feat_num, 
           loss='mean_absolute_error')
net.fit(x_transf, 
        y_transf,
        epochs=300, 
        batch_size=int(x_transf.shape[0]/8), 
        verbose=0)

#### Below we create a function that transfers hiddel layers (up to index 'idx', starting from 0) to a new network. 'lhl_sizes' is a tuple defining the number of neurons in each hidden layer, e.g. (200,100) means two hidden layers with 200 and 100 neurons respectivelly.

In [127]:
def transf_MLP(feat_num, idx, lhl_sizes, loss='mean_absolute_error'):
    global net  # net is a networks and cannot be pickled! Therefore it cannot be an input argument for cross_validate() to work!
    inp = Input(shape=(feat_num,))
    shared_layer = net.layers[0]
    shared_layer.trainable = False  # deactivate training in all re-used layers of MLP1
    out_tensor = shared_layer(inp)
    # idx = 1  # index of desired layer
    for i in range(1,idx+1):
        shared_layer = net.layers[i]    # deactivate training in all re-used layers of MLP1
        shared_layer.trainable = False  # deactivate training in all re-used layers of MLP1
        out_tensor = shared_layer(out_tensor)
    # Here add all the new layers
    for l_size in lhl_sizes:
        out_tensor = Dense(l_size, kernel_initializer='normal', activation='relu')(out_tensor)
    # Close the network
    out_tensor = Dense(1, kernel_initializer='normal')(out_tensor)
    # Create the model
    transf_model = Model(inp, out_tensor)
    transf_model.compile(loss=loss, optimizer='adam')
    return transf_model

In [128]:
estimator = KerasRegressor(build_fn=transf_MLP,
                           feat_num=feat_num,
                           idx=0,
                           lhl_sizes=(300, 200),
                           loss='mean_absolute_error',
                           epochs=300,
                           batch_size=int(x.shape[0]/8),
                           verbose=0)

#### Measure the performance of the new hybrid network on our initial small dataset with 5-fold cross-validation.

In [129]:
scores = cross_validate(estimator, x, y, scoring=scoring, cv=kfold, return_train_score=False)
print (scores)
print("\nResults: average tau=%f+-%f, average R=%f+-%f, average MSE=%f+-%f\n" %
      (scores['test_tau'].mean(), scores['test_tau'].std(),
       scores['test_R'].mean(), scores['test_R'].std(),
       scores['test_MSE'].mean(), scores['test_MSE'].std()))

{'score_time': array([0.61949897, 0.6279192 , 0.63407707, 0.63969994, 0.65007305]), 'test_MSE': array([1.32864658, 1.03584473, 0.80954335, 0.61161483, 1.02622082]), 'fit_time': array([ 9.79497313,  9.34225893, 24.54360104,  9.45413208,  9.42776895]), 'test_R': array([0.57447738, 0.80218974, 0.81670668, 0.88901651, 0.86815936]), 'test_tau': array([0.34607341, 0.62068051, 0.6164606 , 0.71482433, 0.66855008])}

Results: average tau=0.593318+-0.128715, average R=0.790110+-0.112450, average MSE=0.962374+-0.240840



#### We see an impressive performance gain!

In [78]:
scores = cross_validate(estimator, x, y, scoring=scoring, cv=kfold, return_train_score=False,)
print scores
print("\nResults: average tau=%f+-%f, average R=%f+-%f, average MSE=%f+-%f\n" %
      (scores['test_tau'].mean(), scores['test_tau'].std(),
       scores['test_R'].mean(), scores['test_R'].std(),
       scores['test_MSE'].mean(), scores['test_MSE'].std()))

{'score_time': array([0.35903096, 0.35371709, 0.36846399, 0.36452699, 0.37568092]), 'test_MSE': array([1.16594237, 1.34469876, 0.78113445, 1.14604242, 1.09889925]), 'fit_time': array([9.7914331 , 7.95971394, 8.05258393, 8.59206796, 8.59934092]), 'test_R': array([0.69472628, 0.79244967, 0.8679898 , 0.81822004, 0.8420003 ]), 'test_tau': array([0.50000015, 0.60782221, 0.61140077, 0.65371449, 0.59554902])}

Results: average tau=0.593697+-0.050789, average R=0.803077+-0.059684, average MSE=1.107343+-0.183168



#### NOTE:
Transfer learning does not always improve the performance. In this case we used a larger set of compounds binding to the same receptor (CDK2) to pretrain an network and transfer it's hidden layer to another one. If we have done the same but with compounds binding to CDK1 (59% sequence identity with CDK2) then the performance would have been worse. So be caucious where you apply transfer learning and which training data you use!
As a home work you can apply the same procedure by instead of "data/cdk2_large.sdf" and "data/cdk2_pK.dat", to use "data/cdk1_large.sdf" and "data/cdk1_pK.dat".

## Train a Classifier network instead of a Regressor.

#### The difference with our Regressor MLP1 is that the output layer contains a single neuron and uses the sigmoid activation function in order to produce a probability output in the range of 0 to 1 that can easily and automatically be converted to class values.

In [130]:
# create a Classifier
def MLP3(feat_num, loss='binary_crossentropy'):
    net = Sequential()
    net.add(Dense(300, input_dim=feat_num, kernel_initializer='normal', activation='relu'))
    net.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    net.compile(optimizer='rmsprop', loss=loss, metrics=['accuracy'])
    # Compile model
    net.compile(loss=loss, optimizer='adam')
    return net

#### Load and prepare the Blood Brain Barrier permeability data for classification.

In [140]:
fname = "data/logBB.sdf"

mols = []
y = []
for mol in SDMolSupplier(fname):
    if mol is not None:
        mols.append(mol)
        y.append(float(mol.GetProp("logBB_class")))

#### Generate binary Morgan fingerprint with radius 2 for training.

In [141]:
fp = [AllChem.GetMorganFingerprintAsBitVect(m, 2) for m in mols]

In [142]:
x = rdkit_numpy_convert(fp)

In [143]:
mol_num, feat_num = x.shape
print("# molecules for training = %i, # of features = %i\n" % (mol_num, feat_num))

# molecules for training = 321, # of features = 2048



#### Print summary of layers and trainable parameters.

In [144]:
MLP3(feat_num, 'binary_crossentropy').summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_267 (Dense)            (None, 300)               614700    
_________________________________________________________________
dense_268 (Dense)            (None, 1)                 301       
Total params: 615,001
Trainable params: 615,001
Non-trainable params: 0
_________________________________________________________________


In [145]:
estimator = KerasClassifier(build_fn=MLP3, 
                           feat_num=feat_num, 
                           loss='binary_crossentropy', 
                           epochs=100,  # ~300 is the optimum value, but we use 1000 to see the effect of overfitting
                           batch_size=int(x.shape[0]/8), 
                           verbose=0)

#### Use this time a classification metric to score the predictions, the area under the Receiver Operating Characteristic Curve (AUC-ROC).

In [146]:
def AUC_ROC(estimator, X, y):
    preds = estimator.predict(X)
    auc = roc_auc_score(preds, y)
    return auc

scoring = {'roc': AUC_ROC}
kfold = KFold(n_splits=5, random_state=seed)
scores = cross_validate(estimator, x, y, scoring=scoring, cv=kfold, return_train_score=False)
print (scores)
print("\nResults: average AUC-ROC=%f+-%f\n" % 
      (scores['test_roc'].mean(), scores['test_roc'].std()))

{'score_time': array([0.68351793, 0.65033007, 0.65935397, 0.66439486, 0.6893599 ]), 'test_roc': array([0.71388889, 0.66966967, 0.79545455, 0.78615385, 0.61862745]), 'fit_time': array([6.90146089, 6.71428204, 6.75946999, 6.74006104, 6.746907  ])}

Results: average AUC-ROC=0.716759+-0.067623



#### The MLP classifier had a relative good performance. Compare with with ML model performance from the QSAR tutorial.

#### Let's use early stopping to see if the performance improves even further.

In [147]:
scores = cross_validate(estimator, x, y, scoring=scoring, cv=kfold, return_train_score=False,
                        fit_params={'callbacks': [EarlyStopping(patience=3)]})
print (scores)
print("\nResults: average AUC-ROC=%f+-%f\n" % 
      (scores['test_roc'].mean(), scores['test_roc'].std()))

{'score_time': array([0.68597102, 0.68706489, 0.70336699, 0.69680381, 0.71951294]), 'test_roc': array([0.67917548, 0.66966967, 0.81118494, 0.79757085, 0.61862745]), 'fit_time': array([7.01429892, 6.82169008, 6.8170321 , 6.82987309, 6.84853411])}

Results: average AUC-ROC=0.715246+-0.075756



## Try to recreate each of the network architectures that you created with Keras using the following online tool:

#### http://playground.tensorflow.org

#### We don't see statistically significant change, probably because the training set is small (321 molecules). In real case problems you have thousands of training samples and there the effects of overfitting are more evident.